In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 GCP 上进行的 E2E 机器学习：MLOps 阶段 2：实验：开始使用 Vertex AI Training 进行 R 的训练

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在 Google Cloud 上生产环境中使用 Vertex AI 进行端到端 MLOps。本教程涵盖了第二阶段：实验阶段：使用 Vertex AI Training for R 入门。请注意，此笔记本应仅在 R 笔记本图像（例如，R4.1）中运行。

### 目标

在本教程中，您将学习如何使用`Vertex AI Training`来训练一个自定义的 R 模型。

此教程使用以下谷歌云 ML 服务:

- `Vertex AI Training`
- `Vertex AI Model`资源

执行的步骤包括:

- 在笔记本中使用 %%R 魔术命令本地训练一个 R 模型
- 创建一个包含训练好的 R 模型和服务功能的部署镜像
- 在本地测试部署镜像
- 为带有嵌入式 R 模型的部署镜像创建一个`Vertex AI Model`资源
- 将带有嵌入式 R 模型的部署镜像部署到`Vertex AI Endpoint`资源
- 使用带有嵌入式 R 模型的部署镜像进行测试
- 创建一个从 R 到 Python 的训练包
- 为训练模型创建一个训练镜像
- 使用带有 R 到 Python 训练包的 `Vertex AI Training` 服务来训练一个 R 模型

### 数据集

本教程使用的数据集是内置于 R 包中的鸢尾花数据集。该数据集不需要任何特征工程。训练的模型预测鸢尾花的类型，属于三种品种中的一种：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装步骤

安装执行这个笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG -q
! pip3 install --upgrade rpy2 $USER_FLAG -q

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它能找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得一笔 300 美元的免费信用额用于支付计算/存储成本。

1. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在此笔记本的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是支持Vertex AI的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}
if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上出现名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务账号**。

2. 在**服务账号名称**字段中输入名称，并点击**创建**。

3. 在**向此服务账号授予权限给项目**部分，点击角色下拉列表。在筛选框中输入“Vertex AI”，选择**Vertex AI管理员**。在筛选框中输入“存储对象管理员”，选择**存储对象管理员**。

4. 点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境中。

5. 在下面的单元格中以GOOGLE_APPLICATION_CREDENTIALS变量的形式输入您的服务账号密钥的路径，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要按照以下步骤操作。**

在初始化用于Python的Vertex AI SDK时，您需要指定一个云存储临时存储桶。这个临时存储桶是您数据集和模型资源相关数据在会话间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称在所有谷歌云项目中必须是全局唯一的，包括您组织之外的谷歌云项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import traceback

import google.cloud.aiplatform as aip

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`以在CPU上运行容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3之前的GPU支持版本在加载此教程中的自定义模型时将无法运行。这是一个已知问题，在TF 2.3中已修复。这是由于在服务函数中生成静态图操作。如果在自己的自定义模型中遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个CPU核心

*注意：以下不支持用于训练:*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## R培训简介

### 训练一个R模型

您可以在本地训练模型，也可以使用`Vertex AI Training`服务。在后一种情况下，您需要将R到Python解析器`rpy2`安装到您的训练实例（例如，setup.py），并使用R到Python解析器`rpy2`执行R训练脚本。

### 部署一个R模型

在`Vertex AI Prediction`服务上部署一个R模型需要使用一个定制的容器来提供在线预测。在本教程中，您将部署一个运行plumber R包的容器，以提供来自训练模型工件的预测。

### 载入 R 笔记本解释器

载入模块 `rpy2.ipython` 将在您的笔记本中添加对 %R 和 %RR 魔法单元的支持。

In [ ]:
%load_ext rpy2.ipython

### 安装和引入一些额外的R包

In [ ]:
%%R

install.packages(c("randomForest", "plumber"), repos = "http://cran.us.r-project.org")

library(ggplot2)
library(randomForest)
library(plumber)

快速查看您的数据

本教程使用了内置在R软件包中的鸢尾花数据集的版本。

首先，快速查看数据。

In [ ]:
%%R

head(iris)

In [ ]:
# Make folder for R
! rm -rf deploy custom
! mkdir deploy custom custom/trainer

### 在本地训练一个R模型

首先，您在本地训练这个R模型。

In [ ]:
%%R

# train model
model = randomForest(Species ~ ., data = iris)
# save model
save(model, file = "deploy/model.RData")

### 提供脚本

您创建了一个用于提供预测的 R 脚本。此脚本执行以下操作：

- 从输入请求的 HTTP 主体中提取预测请求数值。
- 为 R 模型构建一个预测请求。
- 将预测请求提交给 R 模型。
- 返回预测结果。

*注意*：Plumber 使用注释“注解”在函数上面定义 Web 服务。当您将文件输入 Plumber 时，您将获得一个可运行的 Web 服务，其他系统可以通过网络与之交互。

In [ ]:
%%writefile deploy/serving.R
# serving.R

library("randomForest")

#* Health check
#* @get /ping
#* @serializer unboxedJSON
function() {
    list(status = "OK")
}

#* @apiTitle flower classifier
#* @param petal_length
#* @param petal_width
#* @param sepal_length
#* @param sepal_width
#* @post /classify
function (req)
{
    instances <- as.data.frame(jsonlite::fromJSON(req$postBody))
    results <- list()

    load("./model.RData")

    for(i in 1:nrow(instances)) {       # for-loop over columns
        petal_length <- instances[i, "instances.petal_length"]
        petal_width <- instances[i, "instances.petal_width"]
        sepal_length <- instances[i, "instances.sepal_length"]
        sepal_width <- instances[i, "instances.sepal_width"]
        test = c(sepal_length, sepal_width, petal_length, petal_width)
        test = sapply(test, as.numeric)
        test = data.frame(matrix(test, ncol = 4))
        colnames(test) = colnames(iris[, 1:4])
        results <- append(results, predict(model, test))
    }

    list(predictions = results)
}

#### 运行R服务器的脚本

接下来，您需要创建一个运行服务器的文件。

In [ ]:
%%writefile deploy/startServer.R
library(plumber)
pr <- plumb("serving.R")
pr$run(host = "0.0.0.0", port = 7080)

### 为预测创建 R 容器

目前，Vertex AI 没有预定义的容器可以用于部署 R 模型进行预测。没关系，您可以自己组装一个定制的容器。在本教程中，您可以按照以下步骤从一个 Docker 镜像构建一个部署容器：

- 设置由 RStudio 提供的基础镜像（rstudio/plumber）。
- 将模型工件和服务脚本打包到一个 Docker 镜像中。
- 在端口 7080 上启动服务脚本。

In [ ]:
%%writefile deploy/Dockerfile

FROM rstudio/plumber

# install random forest
RUN R -e 'install.packages(c("randomForest"), repos = "https://cran.rstudio.com/")'

# Copy model and script
RUN mkdir /app
COPY model.RData /app
COPY serving.R /app
COPY startServer.R /app
WORKDIR /app

# plumber & run server
EXPOSE 7080

ENTRYPOINT ["R", "-f", "/app/startServer.R"]

In [ ]:
DEPLOY_IMAGE = f"gcr.io/{PROJECT_ID}/r-predict-iris"
print(DEPLOY_IMAGE)

! docker build --tag=$DEPLOY_IMAGE ./deploy

! docker push $DEPLOY_IMAGE

### 本地测试Docker镜像

接下来，您可以本地测试已经创建好的用于提供预测服务的Docker镜像。

#### 启动服务二进制文件

首先，启动服务二进制文件，监听端口7080，并进行健康检查。

In [ ]:
! docker stop local_iris 2>/dev/null
! docker run -t -d --rm -p 7080:7080 --name=local_iris $DEPLOY_IMAGE
! sleep 10
! curl http://localhost:7080/ping

发送预测请求

接下来，您将向您本地启动的服务器二进制文件发送一个预测请求。之后，您将关闭启动的服务器二进制文件。

In [ ]:
%%bash

cat > ./deploy/instances.json <<END
{
  "instances": [{
      "sepal_width": 1,
      "sepal_length": 2,
      "petal_width": 3,
      "petal_length": 1
    },
    {
      "sepal_width": 4,
      "sepal_length": 2,
      "petal_width": 1,
      "petal_length": 1
    }
  ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./deploy/instances.json \
  http://localhost:7080/classify

docker stop local_iris 1>/dev/null

### 将一个 R 模型上传至 `Vertex AI Model` 资源

接下来，将 R 模型上传到 Vertex AI Model 资源，具有以下参数：

- `display_name`：模型资源的可读名称。
- `serving_container_image_uri`：包含部署二进制文件和 R 模型的部署镜像。
- `serving_container_predict_route`：用于预测请求的 URI 端点。
- `serving_container_health_route`：用于健康检测 ping 的 URI 端点。
- `serving_container_ports`：用于监听预测/健康请求的端口列表。

In [ ]:
DISPLAY_NAME = "iris_" + TIMESTAMP
health_route = "/ping"
predict_route = "/classify"
serving_container_ports = [7080]

model = aip.Model.upload(
    display_name=DISPLAY_NAME,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

## 部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：端点上流量的百分比将流向该模型，可以指定为一个或多个键/值对的字典。
如果只有一个模型，那么指定为{ "0": 100 }，其中"0"是指上传的这个模型，100表示100%的流量。
如果端点上存在其他模型，需要分配流量，则使用model_id来指定为{ "0": percent, model_id: percent, ... }，其中model_id是已部署到端点的现有模型的模型ID。这些百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `starting_replica_count`：最初要预留的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，只预留一个实例。

In [ ]:
DEPLOYED_NAME = "iris-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### 进行测试预测

接下来，您可以通过发送合成数据来测试已部署的模型。

In [ ]:
try:
    INSTANCES = [
        {"sepal_width": 1, "sepal_length": 2, "petal_width": 3, "petal_length": 1},
        {"sepal_width": 4, "sepal_length": 2, "petal_width": 1, "petal_length": 1},
    ]

    prediction = endpoint.predict(instances=INSTANCES)

    print(prediction)
except:
    traceback.print_exc()

撤销模型

当您完成预测后，您可以从“端点”资源中撤销模型。这将取消所有计算资源，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除端点

方法“delete()”将删除端点。

In [ ]:
endpoint.delete()

删除模型

方法'delete（）'将会删除模型。

In [ ]:
model.delete()

### 为 Python/R 培训包创建任务脚本

接下来，您需要为培训包创建 `task.py` 脚本。一些重要的步骤包括：

- 命令行参数：
    - `model-dir`：保存训练模型的位置。在使用 Vertex AI 自定义训练时，该位置将在环境变量 `AIP_MODEL_DIR` 中指定。

- 培训：
    - 使用 R 到 Python 解释器来运行 R 培训脚本。

- 模型工件保存：
    - 在由 `model-dir` 指定的 Cloud 存储位置保存模型工件。
    - *注意*：GCSFuse（`/gcs`）用于在 Cloud 存储存储桶上执行文件系统操作。

In [ ]:
%%writefile custom/trainer/task.py

import os
import sys
import rpy2
import argparse
import logging

# import rpy2's package module
import rpy2.robjects.packages as rpackages

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

# import R's utility package
randomForest = rpackages.importr('randomForest')

# train the model
logging.info("Model training started ...")
rpy2.robjects.r('''
    # train model
    model = randomForest(Species ~ ., data = iris)
    # save model
    save(model, file = "model.RData")
'''
)
logging.info("Model training completed ...")

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, 'model.RData')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
with open("model.RData", "rb") as f:
    data = f.read()
with open(gcs_model_path, "wb") as f:
    f.write(data)

为训练创建 R 容器

目前，Vertex AI 没有用于训练 R 模型的预定义容器。没问题，您可以自己组装自定义容器。在本教程中，您可以按照以下步骤从 Docker 镜像构建部署容器：

- 将基本镜像设置为 TensorFlow 深度学习镜像
- 安装 R-to-Python 包和其他 R 工具
- 安装云存储包
- 将模型内容和服务脚本打包到 Docker 镜像中。
- 在容器中设置入口点以运行训练包

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3

RUN apt-get update && apt-get install -y --no-install-recommends build-essential r-base r-cran-randomforest python3.6 python3-pip python3-setuptools python3-dev
RUN pip install google-cloud-storage
RUN pip install rpy2

# install random forest
RUN R -e 'install.packages(c("randomForest"), repos = "https://cran.rstudio.com/")'

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

In [ ]:
TRAIN_IMAGE = f"gcr.io/{PROJECT_ID}/r-train-iris"
print(TRAIN_IMAGE)

! docker build --tag=$TRAIN_IMAGE ./custom

! docker push $TRAIN_IMAGE

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `command`：在容器内调用的命令（例如解释器）和脚本。

*注意：* 在容器内可以覆盖命令和脚本的调用方式（即 ENTRYPOINT）。

In [ ]:
job = aip.CustomContainerTrainingJob(
    display_name="iris_" + TIMESTAMP,
    container_uri=TRAIN_IMAGE,
    command=["python3", "trainer/task.py"],
)

print(job)

运行自定义容器训练任务

接下来，通过调用`run()`方法运行自定义任务来启动训练任务。参数与运行自定义训练任务时相同。

In [ ]:
CMDARGS = ["--model-dir=" + BUCKET_URI]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

删除自定义训练作业

训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

清理

要清理本项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源：

- 模型（已在先前单元格中删除）
- 端点（已在先前单元格中删除）
- 自定义作业（已在先前单元格中删除）
- 云存储桶

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI